In [1]:
import pandas as pd
import numpy as np
from utils.Data import Data
from utils.Camera import Camera
from utils.Plot import Plot

In [2]:
data = Data()
camera = Camera('data/camera.dat')

INFO - Loading trajectory data from data//trajectory.dat
INFO - 0.00 [s] - Trajectory data loaded successfully!
INFO - Loading world data from data//world.dat
INFO - 0.00 [s] - World data loaded successfully!
INFO - Loading measurements data from data//meas-*.dat
INFO - 0.04 [s] - Measurements data loaded successfully!
INFO - Loading camera parameters from data/camera.dat
INFO - 0.00s - Camera parameters loaded successfully.


In [3]:
def compute_points(measurement):
    gt_pose = measurement['gt_pose']
    odom_pose = measurement['odom_pose']
    points = {'point_id':[], 'image_point':[], 'camera_point':[], 'world_point':[], 'appearance':[]}  

    for point in measurement['points']:    

        points['point_id'].append(measurement['points'][point]['actual_point_id'])

        image_point = measurement['points'][point]['image_point']
        points['image_point'].append(image_point)

        camera_point = camera.pixel_to_camera(image_point)
        points['camera_point'].append(camera_point)

        world_point = camera.camera_to_world(camera_point)
        points['world_point'].append(world_point)

        points['appearance'].append(measurement['points'][point]['appearance'])

    return gt_pose, odom_pose, points

In [4]:
def data_association(points_1, points_2):
    matches = dict()
    for i in range(len(points_1['point_id'])):
        for j in range(len(points_2['point_id'])):
            if points_1['appearance'][i] == points_2['appearance'][j]:
                id = points_1['point_id'][i]
                world_point_1 = points_1['world_point'][i]
                world_point_2 = points_2['world_point'][j]
                matches[id] = [world_point_1, world_point_2]
    return matches

In [5]:
def compute_R_t(set1, set2):

    '''
    Given two sets of points, set1 and set2, which are the corresponding points in two different coordinate systems,
    this function returns the rotation and translation matrix that best aligns the two sets of points.
    '''

    # Compute the centroid of each set of points
    x_0 = np.mean(set1, axis=0)
    y_0 = np.mean(set2, axis=0)

    # Subtract the centroid from each set of points
    set1 = set1 - x_0
    set2 = set2 - y_0

    # Calculate the covariance matrix of the two sets of points
    H = np.dot(set1.T, set2)
    
    # Use the SVD to calculate the rotation matrix
    U, S, Vt = np.linalg.svd(H)

    # Calculate the rotation matrix
    R = np.dot(Vt.T, U.T)

    # Calculate the translation matrix
    T = y_0 - np.dot(R, x_0)

    return R, T

In [6]:
data.print_trajectory(0)

Trajectory data: 
pose_id: 0
  ground_truth_pose: [0.0, 0.0, 0.0]
  odometry_pose:     [0.00160159, 0.0, -0.000259093]


In [7]:
data.print_world(6)

World data: 
landmark_id: 6
  landmark_position:   [2.79958, -2.91903, 0.751446]
  landmark_appearance: [-0.668052, -0.119791, 0.76015, 0.658402, -0.339326, -0.542064, 0.786745, -0.29928, 0.37334, 0.912936]


In [8]:
data.print_measurements(0)

Measurements data: 
sequence_id: 0
  ground_truth_pose: [0.0, 0.0, 0.0]
  odometry_pose:     [0.00160159, 0.0, -0.000259093]
  Points: 
    point_id: 0
      actual_point_id: 6
      image_point:     [522.119, 187.968]
      appearance:      [-0.668052, -0.119791, 0.76015, 0.658402, -0.339326, -0.542064, 0.786745, -0.29928, 0.37334, 0.912936]
    point_id: 1
      actual_point_id: 14
      image_point:     [442.949, 142.838]
      appearance:      [-0.80072, 0.0616159, 0.514588, -0.39141, 0.984457, 0.153942, 0.755228, 0.495619, 0.25782, -0.929158]
    point_id: 2
      actual_point_id: 15
      image_point:     [67.7619, 175.604]
      appearance:      [0.746543, 0.662075, 0.958868, 0.487622, 0.806733, 0.967191, 0.333761, -0.00548297, -0.672064, 0.660024]
    point_id: 3
      actual_point_id: 17
      image_point:     [306.371, 119.738]
      appearance:      [-0.439916, 0.0922137, 0.438537, -0.773439, -0.0570331, 0.18508, 0.888636, -0.0981649, -0.327298, 0.695369]
    point_id: 4
   

In [9]:
gt1, odom1, points1 = compute_points(data.get_measurements(0))
points1_df = pd.DataFrame(points1)
points1_df

,point_id,image_point,camera_point,world_point,appearance
0,6,"[522.119, 187.968]","[-1.7742687673611113, -1.3321288888888891, 1.0]","[1.2, 1.7742687673611113, 1.3321288888888891]","[-0.668052, -0.119791, 0.76015, 0.658402, -0.3..."
1,14,"[442.949, 142.838]","[-1.7756432465277778, -1.3310842129629632, 1.0]","[1.2, 1.7756432465277778, 1.3310842129629632]","[-0.80072, 0.0616159, 0.514588, -0.39141, 0.98..."
2,15,"[67.7619, 175.604]","[-1.7821569114583335, -1.3318426851851852, 1.0]","[1.2, 1.7821569114583335, 1.3318426851851852]","[0.746543, 0.662075, 0.958868, 0.487622, 0.806..."
3,17,"[306.371, 119.738]","[-1.7780143923611111, -1.330549490740741, 1.0]","[1.2, 1.7780143923611111, 1.330549490740741]","[-0.439916, 0.0922137, 0.438537, -0.773439, -0..."
4,35,"[509.615, 216.308]","[-1.7744858506944445, -1.3327849074074076, 1.0]","[1.2, 1.7744858506944445, 1.3327849074074076]","[0.104633, 0.839182, 0.371973, 0.619571, 0.395..."
...,...,...,...,...,...
122,942,"[563.308, 146.283]","[-1.7735536805555556, -1.3311639583333335, 1.0]","[1.2, 1.7735536805555556, 1.3311639583333335]","[-0.770661, -0.313814, 0.627311, -0.489216, 0...."
123,963,"[302.141, 194.465]","[-1.7780878298611111, -1.3322792824074075, 1.0]","[1.2, 1.7780878298611111, 1.3322792824074075]","[-0.0250311, -0.80992, -0.385503, 0.413624, -0..."
124,971,"[340.454, 198.249]","[-1.7774226736111112, -1.3323668750000002, 1.0]","[1.2, 1.7774226736111112, 1.3323668750000002]","[0.683682, -0.00590533, 0.276129, -0.692478, 0..."
125,979,"[110.052, 55.203]","[-1.7814227083333334, -1.329055625, 1.0]","[1.2, 1.7814227083333334, 1.329055625]","[0.0400809, 0.470501, -0.1505, 0.457585, 0.206..."


In [10]:
gt2, odom2, points2 = compute_points(data.get_measurements(1))
points2_df = pd.DataFrame(points2)
points2_df

,point_id,image_point,camera_point,world_point,appearance
0,6,"[539.005, 183.622]","[-1.773975607638889, -1.3320282870370372, 1.0]","[1.2, 1.773975607638889, 1.3320282870370372]","[-0.668052, -0.119791, 0.76015, 0.658402, -0.3..."
1,14,"[463.689, 126.449]","[-1.7752831770833335, -1.3307048379629631, 1.0]","[1.2, 1.7752831770833335, 1.3307048379629631]","[-0.80072, 0.0616159, 0.514588, -0.39141, 0.98..."
2,15,"[56.6646, 172.771]","[-1.7823495729166667, -1.3317771064814816, 1.0]","[1.2, 1.7823495729166667, 1.3317771064814816]","[0.746543, 0.662075, 0.958868, 0.487622, 0.806..."
3,17,"[305.329, 110.546]","[-1.778032482638889, -1.330336712962963, 1.0]","[1.2, 1.778032482638889, 1.330336712962963]","[-0.439916, 0.0922137, 0.438537, -0.773439, -0..."
4,35,"[518.356, 215.216]","[-1.7743340972222224, -1.3327596296296298, 1.0]","[1.2, 1.7743340972222224, 1.3327596296296298]","[0.104633, 0.839182, 0.371973, 0.619571, 0.395..."
...,...,...,...,...,...
119,939,"[263.409, 211.305]","[-1.7787602604166668, -1.3326690972222224, 1.0]","[1.2, 1.7787602604166668, 1.3326690972222224]","[-0.426205, -0.741321, 0.567446, 0.887447, 0.2..."
120,963,"[300.962, 191.459]","[-1.7781082986111112, -1.3322096990740742, 1.0]","[1.2, 1.7781082986111112, 1.3322096990740742]","[-0.0250311, -0.80992, -0.385503, 0.413624, -0..."
121,971,"[341.861, 195.379]","[-1.777398246527778, -1.332300439814815, 1.0]","[1.2, 1.777398246527778, 1.332300439814815]","[0.683682, -0.00590533, 0.276129, -0.692478, 0..."
122,979,"[81.2067, 29.8137]","[-1.781923494791667, -1.3284679097222223, 1.0]","[1.2, 1.781923494791667, 1.3284679097222223]","[0.0400809, 0.470501, -0.1505, 0.457585, 0.206..."


In [11]:
matches = data_association(points1, points2)
matches_df = pd.DataFrame(matches).T
matches_df.columns = ['world_point_1', 'world_point_2']
matches_df

,world_point_1,world_point_2
6,"[1.2, 1.7742687673611113, 1.3321288888888891]","[1.2, 1.773975607638889, 1.3320282870370372]"
14,"[1.2, 1.7756432465277778, 1.3310842129629632]","[1.2, 1.7752831770833335, 1.3307048379629631]"
15,"[1.2, 1.7821569114583335, 1.3318426851851852]","[1.2, 1.7823495729166667, 1.3317771064814816]"
17,"[1.2, 1.7780143923611111, 1.330549490740741]","[1.2, 1.778032482638889, 1.330336712962963]"
35,"[1.2, 1.7744858506944445, 1.3327849074074076]","[1.2, 1.7743340972222224, 1.3327596296296298]"
...,...,...
939,"[1.2, 1.7787169270833334, 1.332698402777778]","[1.2, 1.7787602604166668, 1.3326690972222224]"
963,"[1.2, 1.7780878298611111, 1.3322792824074075]","[1.2, 1.7781082986111112, 1.3322096990740742]"
971,"[1.2, 1.7774226736111112, 1.3323668750000002]","[1.2, 1.777398246527778, 1.332300439814815]"
979,"[1.2, 1.7814227083333334, 1.329055625]","[1.2, 1.781923494791667, 1.3284679097222223]"


In [12]:
R = np.eye(3)
T = np.zeros(3)
estimated_trajectory = []

for i in range(120):
    measurement_1 = data.get_measurements(i)
    measurement_2 = data.get_measurements(i+1)

    gt_pose_1, odom_pose_1, points_1 = compute_points(measurement_1)
    gt_pose_2, odom_pose_2, points_2 = compute_points(measurement_2)

    matches = data_association(points_1, points_2)

    set1 = np.array([matches[key][0] for key in matches])
    set2 = np.array([matches[key][1] for key in matches])

    estimated_R, estimated_T = compute_R_t(set1, set2)

    R = np.dot(R, estimated_R)
    T = T + np.dot(R, estimated_T)

    updated_pose = np.round(np.dot(estimated_R, odom_pose_1) + estimated_T, 8)

    estimated_trajectory.append(updated_pose.tolist())

In [13]:
trajectory_data = data.get_trajectory()
gt_trajectory = []
odom_trajectory = []
for key, value in trajectory_data.items():
    gt_trajectory.append(value['ground_truth_pose'])
    odom_trajectory.append(value['odometry_pose'])

In [14]:
print(f'Ground Truth Trajectory: \n {gt_trajectory}')
print(f'Odometry Trajectory: \n {odom_trajectory}')
print(f'Estimated Trajectory: \n {list(estimated_trajectory)}')

Ground Truth Trajectory: 
 [[0.0, 0.0, 0.0], [0.200426, 0.0, 0.0], [0.40085, 0.0, 0.0], [0.601274, 0.0, 0.0], [0.801698, 0.0, 0.0], [1.00244, 0.0, 0.0], [1.20254, 0.0, 0.0], [1.40263, 0.0, 0.0], [1.60272, 0.0, 0.0], [1.80282, 0.0, 0.0], [2.00291, 0.0, 0.0], [2.20353, 0.0, 0.0], [2.40355, 0.0, 0.0], [2.60358, 0.0, 0.0], [2.8036, 0.0, 0.0], [3.00362, 0.0, 0.0], [3.20364, 0.0, 0.0], [3.40319, 0.0151316, 0.188353], [3.56783, 0.0638916, 0.388423], [3.71949, 0.144398, 0.58849], [3.85213, 0.253439, 0.788559], [3.96045, 0.386664, 0.988628], [4.04013, 0.538759, 1.1887], [4.08799, 0.703654, 1.38877], [4.1131, 0.902461, 1.45456], [4.13633, 1.10141, 1.45456], [4.15956, 1.30036, 1.45456], [4.18278, 1.4993, 1.45456], [4.20601, 1.69824, 1.45456], [4.22923, 1.89719, 1.45456], [4.25246, 2.09613, 1.45456], [4.27568, 2.29488, 1.45456], [4.29899, 2.49425, 1.45456], [4.32229, 2.69361, 1.45456], [4.34559, 2.89298, 1.45456], [4.36889, 3.09234, 1.45456], [4.39219, 3.29171, 1.45456], [4.41549, 3.49107, 1.45456

In [15]:
max_points = 120
estimated_trajectory_to_plot = estimated_trajectory[:max_points]
gt_trajectory_to_plot = gt_trajectory[:max_points]
odom_trajectory_to_plot = odom_trajectory[:max_points]

In [17]:
plot = Plot(title='Trajectories: Ground Truth, Odometry and Estimated')
plot.add_trajectory(gt_trajectory_to_plot, 'Ground Truth', 'green', 2)
plot.add_trajectory(odom_trajectory_to_plot, 'Odometry', 'blue', 2)
plot.add_trajectory(estimated_trajectory_to_plot, 'Estimated', 'red', 2)
plot.show()